In [64]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [65]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [66]:
tf.keras.backend.clear_session()

In [67]:
subject = 'Maranhão - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [68]:
data = pd.read_csv('2003_mo_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003-1,23.368400,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.213662,0.260097,0.635570,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,43.342
1,2003-2,23.167977,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.207986,0.258964,0.635997,4.320832e+07,2.636851e+06,6.036083,3.874405e+07,28.663
2,2003-3,23.563273,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.202310,0.257352,0.636424,4.326343e+07,2.637701e+06,6.038110,3.876316e+07,27.900
3,2003-4,24.008345,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.196633,0.255929,0.636851,4.331853e+07,2.638551e+06,6.040136,3.878226e+07,25.739
4,2003-5,24.366896,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.190957,0.254109,0.637278,4.337364e+07,2.639401e+06,6.042163,3.880136e+07,30.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,31.724066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,102.698
236,2022-9,31.679231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,98.133
237,2022-10,31.524870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,94.964
238,2022-11,31.512212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,97.684


In [69]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.189788,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.185037,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.180287,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.175536,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [70]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Maranhão - Consumo de Cimento (t), Length: 240, dtype: float64

In [71]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.796451,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.431469,0.944351,1.437372,1.197811,-1.616416,0.972153,1.061498
158,0.727411,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.418921,0.952621,1.436068,1.200726,-1.633804,0.973222,1.061628
159,0.657765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.406373,0.960982,1.434764,1.203641,-1.651193,0.974290,1.061759
160,0.591941,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.393825,0.969363,1.433461,1.206556,-1.668581,0.975359,1.061889


In [72]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
157    66.661
158    78.801
159    67.256
160    84.280
161    86.501
Name: Maranhão - Consumo de Cimento (t), Length: 162, dtype: float64

In [73]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [74]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [75]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
126,1.796771,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.858960,0.673675,0.395980,0.831326,0.271134,1.034804,0.988167
127,1.807797,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.892045,0.687412,0.398649,0.850067,0.230720,1.041385,0.998849
128,1.778040,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.925130,0.706891,0.401317,0.868808,0.190305,1.047966,1.009532
129,1.750228,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.958215,0.722784,0.403986,0.887549,0.149891,1.054547,1.020214
130,1.717721,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.991300,0.738712,0.406655,0.906289,0.109477,1.061127,1.030897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.189788,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.185037,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.180287,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.175536,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [76]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [77]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [78]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [79]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [80]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[615604260, 4134377887, 283210643, 3897998540, 883650697, 3650757850, 1602801949, 3062065704, 852273720, 3096264488, 25125119, 3497327, 4222467103, 701660642, 555819531, 2971849383, 3669791322, 3704099527, 4175035526, 2328610909]


Step: 0 ___________________________________________
val_loss: 135.763427734375
winner_seed: 615604260


Step: 1 ___________________________________________
val_loss: 301.39642333984375


Step: 2 ___________________________________________
val_loss: 63.66977310180664
winner_seed: 283210643


Step: 3 ___________________________________________
val_loss: 95.01361846923828


Step: 4 ___________________________________________
val_loss: 127.73754119873047


Step: 5 ___________________________________________
val_loss: 139.34300231933594


Step: 6 ___________________________________________
val_loss: 154.15591430664062


Step: 7 ___________________________________________
val_loss: 248.57240295410156


Step: 8 ___________________________________________
val_loss: 

In [81]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 131ms/step - loss: 12196.3994 - val_loss: 7761.0649
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 12099.5352 - val_loss: 7746.9712
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 12000.8926 - val_loss: 7725.4409
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 11888.7412 - val_loss: 7694.6172
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 11755.0078 - val_loss: 7657.1650
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 11608.7285 - val_loss: 7605.6831
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 11446.2080 - val_loss: 7534.4536
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 11278.5244 - val_loss: 7439.0308
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 11107.2432 - val_loss: 7331.4688
Epoch 10/10000
4/4 [================

4/4 [==============================] - 0s 10ms/step - loss: 6071.5879 - val_loss: 3423.9905
Epoch 78/10000
4/4 [==============================] - 0s 10ms/step - loss: 6030.1865 - val_loss: 3393.6011
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 5988.9448 - val_loss: 3363.2148
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 5947.5205 - val_loss: 3333.1406
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 5906.9614 - val_loss: 3303.3845
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 5866.2046 - val_loss: 3273.7839
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 5825.9282 - val_loss: 3244.1133
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 5785.9175 - val_loss: 3214.6045
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 5746.2700 - val_loss: 3185.4768
Epoch 86/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 3593.4695 - val_loss: 1639.6057
Epoch 154/10000
4/4 [==============================] - 0s 11ms/step - loss: 3568.8306 - val_loss: 1622.1750
Epoch 155/10000
4/4 [==============================] - 0s 11ms/step - loss: 3544.0930 - val_loss: 1605.0780
Epoch 156/10000
4/4 [==============================] - 0s 11ms/step - loss: 3519.5771 - val_loss: 1588.2837
Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 3495.3049 - val_loss: 1571.7673
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 3471.6416 - val_loss: 1555.2233
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 3447.5320 - val_loss: 1538.9976
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 3423.6267 - val_loss: 1522.5638
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 3399.9543 - val_loss: 1506.2185
Epoch 162/10000
4/4 [=======================

4/4 [==============================] - 0s 12ms/step - loss: 2129.6631 - val_loss: 678.0624
Epoch 230/10000
4/4 [==============================] - 0s 11ms/step - loss: 2115.8354 - val_loss: 669.4819
Epoch 231/10000
4/4 [==============================] - 0s 11ms/step - loss: 2101.8877 - val_loss: 660.9220
Epoch 232/10000
4/4 [==============================] - 0s 12ms/step - loss: 2087.6362 - val_loss: 652.6292
Epoch 233/10000
4/4 [==============================] - 0s 11ms/step - loss: 2073.7837 - val_loss: 644.3448
Epoch 234/10000
4/4 [==============================] - 0s 10ms/step - loss: 2060.1746 - val_loss: 636.0943
Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 2046.2993 - val_loss: 627.8932
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 2032.9725 - val_loss: 619.7153
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 2019.1803 - val_loss: 611.7515
Epoch 238/10000
4/4 [==============================] 

4/4 [==============================] - 0s 10ms/step - loss: 1299.4830 - val_loss: 220.4999
Epoch 306/10000
4/4 [==============================] - 0s 10ms/step - loss: 1290.1198 - val_loss: 217.2122
Epoch 307/10000
4/4 [==============================] - 0s 10ms/step - loss: 1275.1406 - val_loss: 215.6957
Epoch 308/10000
4/4 [==============================] - 0s 10ms/step - loss: 1267.2223 - val_loss: 213.1468
Epoch 309/10000
4/4 [==============================] - 0s 10ms/step - loss: 1260.6877 - val_loss: 211.0572
Epoch 310/10000
4/4 [==============================] - 0s 10ms/step - loss: 1249.6490 - val_loss: 209.2203
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 1241.8042 - val_loss: 206.8665
Epoch 312/10000
4/4 [==============================] - 0s 10ms/step - loss: 1232.7677 - val_loss: 205.1535
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 1224.2189 - val_loss: 201.8043
Epoch 314/10000
4/4 [==============================] 

...

Epoch 769/10000
4/4 [==============================] - 0s 11ms/step - loss: 214.0702 - val_loss: 128.6716
Epoch 770/10000
4/4 [==============================] - 0s 10ms/step - loss: 213.9550 - val_loss: 114.8841
Epoch 771/10000
4/4 [==============================] - 0s 10ms/step - loss: 212.6465 - val_loss: 117.9469
Epoch 772/10000
4/4 [==============================] - 0s 11ms/step - loss: 212.0398 - val_loss: 106.7188
Epoch 773/10000
4/4 [==============================] - 0s 10ms/step - loss: 211.1110 - val_loss: 99.8654
Epoch 774/10000
4/4 [==============================] - 0s 10ms/step - loss: 210.8121 - val_loss: 102.3076
Epoch 775/10000
4/4 [==============================] - 0s 10ms/step - loss: 210.6478 - val_loss: 99.0110
Epoch 776/10000
4/4 [==============================] - 0s 10ms/step - loss: 209.5786 - val_loss: 92.1917
Epoch 777/10000
4/4 [==============================] - 0s 11ms/step - loss: 209.5625 - val_loss: 94.9752
Epoch 778/10000
4/4 [=============================

4/4 [==============================] - 0s 9ms/step - loss: 174.6382 - val_loss: 122.5905
Epoch 848/10000
4/4 [==============================] - 0s 10ms/step - loss: 173.9282 - val_loss: 120.6904
Epoch 849/10000
4/4 [==============================] - 0s 9ms/step - loss: 172.4021 - val_loss: 125.7497
Epoch 850/10000
4/4 [==============================] - 0s 9ms/step - loss: 171.5919 - val_loss: 129.6011
Epoch 851/10000
4/4 [==============================] - 0s 8ms/step - loss: 171.7244 - val_loss: 133.4256
Epoch 852/10000
4/4 [==============================] - 0s 9ms/step - loss: 170.9628 - val_loss: 125.5494
Epoch 853/10000
4/4 [==============================] - 0s 9ms/step - loss: 170.2608 - val_loss: 123.6169
Epoch 854/10000
4/4 [==============================] - 0s 9ms/step - loss: 169.6982 - val_loss: 128.5640
Epoch 855/10000
4/4 [==============================] - 0s 9ms/step - loss: 169.3745 - val_loss: 134.4356
Epoch 856/10000
4/4 [==============================] - 0s 9ms/step - l

Epoch 925/10000
4/4 [==============================] - 0s 10ms/step - loss: 153.5288 - val_loss: 191.6429
Epoch 926/10000
4/4 [==============================] - 0s 10ms/step - loss: 153.0180 - val_loss: 188.5949
Epoch 927/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.9692 - val_loss: 192.2163
Epoch 928/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.8265 - val_loss: 195.0702
Epoch 929/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.6856 - val_loss: 196.4203
Epoch 930/10000
4/4 [==============================] - 0s 11ms/step - loss: 152.5472 - val_loss: 211.6461
Epoch 931/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.3320 - val_loss: 191.2771
Epoch 932/10000
4/4 [==============================] - 0s 11ms/step - loss: 152.0198 - val_loss: 175.0836
Epoch 933/10000
4/4 [==============================] - 0s 10ms/step - loss: 151.5220 - val_loss: 169.2421
Epoch 934/10000
4/4 [=========================

4/4 [==============================] - 0s 10ms/step - loss: 139.5769 - val_loss: 97.2143
Epoch 1003/10000
4/4 [==============================] - 0s 10ms/step - loss: 141.2988 - val_loss: 86.9274
Epoch 1004/10000
4/4 [==============================] - 0s 10ms/step - loss: 138.4821 - val_loss: 73.0524
Epoch 1005/10000
4/4 [==============================] - 0s 10ms/step - loss: 140.9273 - val_loss: 64.7308
Epoch 1006/10000
4/4 [==============================] - 0s 10ms/step - loss: 138.7985 - val_loss: 66.2676
Epoch 1007/10000
4/4 [==============================] - 0s 9ms/step - loss: 143.4481 - val_loss: 64.5682
Epoch 1008/10000
4/4 [==============================] - 0s 10ms/step - loss: 138.7511 - val_loss: 63.1565
Epoch 1009/10000
4/4 [==============================] - 0s 10ms/step - loss: 139.1810 - val_loss: 70.4984
Epoch 1010/10000
4/4 [==============================] - 0s 10ms/step - loss: 139.6014 - val_loss: 73.8802
Epoch 1011/10000
4/4 [==============================] - 0s 10ms/

Epoch 1080/10000
4/4 [==============================] - 0s 9ms/step - loss: 127.7356 - val_loss: 69.8611
Epoch 1081/10000
4/4 [==============================] - 0s 9ms/step - loss: 133.5948 - val_loss: 68.4867
Epoch 1082/10000
4/4 [==============================] - 0s 9ms/step - loss: 130.9328 - val_loss: 68.0306
Epoch 1083/10000
4/4 [==============================] - 0s 11ms/step - loss: 130.8857 - val_loss: 68.8999
Epoch 1084/10000
4/4 [==============================] - 0s 10ms/step - loss: 129.9579 - val_loss: 69.3569
Epoch 1085/10000
4/4 [==============================] - 0s 10ms/step - loss: 127.0599 - val_loss: 71.0828
Epoch 1086/10000
4/4 [==============================] - 0s 9ms/step - loss: 126.2507 - val_loss: 73.2584
Epoch 1087/10000
4/4 [==============================] - 0s 9ms/step - loss: 123.0270 - val_loss: 74.9319
Epoch 1088/10000
4/4 [==============================] - 0s 9ms/step - loss: 125.1685 - val_loss: 75.3235
Epoch 1089/10000
4/4 [==============================

In [82]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [83]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 18ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,76.955574,70.444252,59.78624,48.418755,34.889839,20.767546,1.53088,-0.058981,-0.901961,-1.363595,-2.11798,-2.321434,-3.07965,-2.644847,-2.039284,-2.061425,-2.090737,-2.359515,-1.672514,-0.638837,-0.045833,0.714232,1.631122,2.433282,3.451078,4.360583,5.446438,6.849641,8.95878,11.159266
Target,88.78,86.195,84.121,88.187,79.514,74.061,64.818,44.254,52.169,53.399,45.364,67.103,61.062,73.445,69.471,83.004,77.332,58.591,70.66,50.796,47.834,45.327,54.032,60.657,67.25,70.773,67.01,81.128,81.737,76.841
Error,11.824425,15.750748,24.334763,39.768242,44.624161,53.293449,63.287121,44.312984,53.070961,54.762592,47.481979,69.424431,64.141647,76.089844,71.510284,85.065422,79.422737,60.950516,72.33252,51.434837,47.879833,44.612766,52.400879,58.223721,63.79892,66.412422,61.563564,74.278358,72.778221,65.68174


In [84]:
display(mae)
display(mape)

56.35047

0.8688133

In [85]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [86]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[306.02914]] - Target[827.965]| =  Error: [[521.9359]]; MAPE:[[0.630384]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[-11.854008]] - Target[752.2110000000002]| =  Error: [[764.065]]; MAPE:[[1.0157589]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[40.22579]] - Target[444.73900000000003]| =  Error: [[404.5132]]; MAPE:[[0.9095519]]


[array([[521.9359]], dtype=float32),
 array([[764.065]], dtype=float32),
 array([[404.5132]], dtype=float32)]

563.5047

0.85189825